In [1]:
import os
import cv2
import pydicom
import numpy as np
import pandas as pd
from pydicom.pixel_data_handlers.util import apply_voi_lut

# Rutas al dataset y archivos
dicom_root = '/Volumes/m2/Memoria/Code/PMM/VinDr-Mammo-Preprocessing/data/processed/subset_datos/images_with_other'
csv_path = '/Volumes/m2/Memoria/Code/PMM/VinDr-Mammo-Preprocessing/data/processed/csv/others_with_image_names.csv'
output_root = '/Volumes/m2/Memoria/Code/PMM/VinDr-Mammo-Preprocessing/data/processed/roi_images/ROICROP8'  # Cambiado a ROICROP7

# Crear carpetas principales (training y test)
train_folder = os.path.join(output_root, 'training')
test_folder = os.path.join(output_root, 'test')

os.makedirs(train_folder, exist_ok=True)
os.makedirs(test_folder, exist_ok=True)

# Crear carpetas de masas, calcificaciones y otros, y sus subcarpetas (benigno, maligno, sospechoso)
for split_folder in [train_folder, test_folder]:
    for category in ['masas', 'calcificaciones', 'otros']:
        for birads_category in ['benigno', 'maligno', 'sospechoso']:
            os.makedirs(os.path.join(split_folder, category, birads_category), exist_ok=True)

# Tamaño objetivo al que queremos llegar para el recorte (299x299 píxeles)
TARGET_SIZE = (299, 299)

# Leer las anotaciones CSV que contienen las coordenadas de la ROI
ss1 = pd.read_csv(csv_path)

# Limpiar las categorías eliminando los caracteres adicionales
ss1['finding_categories'] = ss1['finding_categories'].apply(lambda x: x.strip("[]'"))

# Crear un DataFrame vacío para guardar las nuevas coordenadas
ss1_rescaled = pd.DataFrame(columns=ss1.columns)

# Lista de subcategorías para la categoría 'otros'
otros_categories = [
    'Focal Asymmetry',
    'Architectural Distortion',
    'Asymmetry',
    'Suspicious Lymph Node',
    'Skin Thickening',
    'Global Asymmetry',
    'Nipple Retraction',
    'Skin Retraction'
]

# Función para obtener la ruta del archivo DICOM usando image_id
def get_dicom_path(image_id):
    for study_id in os.listdir(dicom_root):
        study_path = os.path.join(dicom_root, study_id)
        if os.path.isdir(study_path):
            dicom_path = os.path.join(study_path, image_id + '.dicom')
            if os.path.exists(dicom_path):
                return dicom_path
    raise FileNotFoundError(f"No se encontró el archivo DICOM para image_id: {image_id}")

# Función para mapear BIRADS a categorías
def map_birads_to_category(birads):
    if birads in ['BI-RADS 0', 'BI-RADS 1', 'BI-RADS 2', 'BI-RADS 3']:
        return 'benigno'
    elif birads == 'BI-RADS 4':
        return 'sospechoso'
    elif birads in ['BI-RADS 5', 'BI-RADS 6']:
        return 'maligno'
    else:
        print(f"Advertencia: BIRADS desconocido '{birads}'. Asignando a 'benigno'.")
        return 'benigno'

# Función para aplicar el recorte basado en el centro de la ROI, con padding en el lado mayor
def extract_roi(image_name, split, birads, finding_categories, ss1_rescaled):
    # Obtener el image_id (parte antes del guión bajo "_")
    image_id = image_name.split('_')[0]
    dicom_path = get_dicom_path(image_id)

    # Leer la imagen DICOM
    dicom = pydicom.dcmread(dicom_path)
    original_image = dicom.pixel_array

    # Aplicar VOI LUT con prefer_lut=True (priorizando LUT si está presente)
    img_windowed = apply_voi_lut(original_image, dicom, prefer_lut=True)

    # Manejar Photometric Interpretation si es MONOCHROME1 (invertir la imagen)
    photometric_interpretation = dicom.get('PhotometricInterpretation', 'UNKNOWN')
    if photometric_interpretation == 'MONOCHROME1':
        img_windowed = img_windowed.max() - img_windowed
        print(f"Imagen {image_name} tiene Photometric Interpretation: {photometric_interpretation} (invertida)")
    else:
        print(f"Imagen {image_name} tiene Photometric Interpretation: {photometric_interpretation}")

    # Normalizar la imagen para que esté en el rango [0, 255]
    img_windowed = (img_windowed - img_windowed.min()) / (img_windowed.max() - img_windowed.min()) * 255
    img_windowed = img_windowed.astype(np.uint8)

    # Extraer las coordenadas de la ROI desde el CSV utilizando el image_name
    try:
        x1 = int(ss1.loc[ss1['image_name'] == image_name, 'xmin'].values[0])
        y1 = int(ss1.loc[ss1['image_name'] == image_name, 'ymin'].values[0])
        x2 = int(ss1.loc[ss1['image_name'] == image_name, 'xmax'].values[0])
        y2 = int(ss1.loc[ss1['image_name'] == image_name, 'ymax'].values[0])
    except IndexError:
        print(f"Error: No se encontraron coordenadas para la ROI de la imagen {image_name}.")
        return ss1_rescaled

    # Verificar que las coordenadas sean válidas
    x1 = max(0, x1)
    y1 = max(0, y1)
    x2 = min(img_windowed.shape[1], x2)
    y2 = min(img_windowed.shape[0], y2)

    # Calcular el tamaño original del recorte
    width = x2 - x1
    height = y2 - y1

    # Agregar padding del 10% al lado mayor y ajustar el lado menor para que sea cuadrado
    if width > height:
        # El ancho es mayor, agregar padding del 10% al ancho
        padding_x = int(width * 0.1)
        width += padding_x
        new_width = width
        new_height = new_width  # Ajustar el alto para hacer la imagen cuadrada
        padding_y = (new_height - height) // 2
    else:
        # El alto es mayor, agregar padding del 10% al alto
        padding_y = int(height * 0.1)
        height += padding_y
        new_height = height
        new_width = new_height  # Ajustar el ancho para hacer la imagen cuadrada
        padding_x = (new_width - width) // 2

    # Asegurarse de que los nuevos límites no excedan los límites de la imagen original
    x1_new = max(x1 - padding_x, 0)
    x2_new = min(x2 + padding_x, img_windowed.shape[1])
    y1_new = max(y1 - padding_y, 0)
    y2_new = min(y2 + padding_y, img_windowed.shape[0])

    # Recortar la imagen utilizando los nuevos límites ajustados
    crop = img_windowed[y1_new:y2_new, x1_new:x2_new]

    # Verificar si el recorte no está vacío
    if crop.size == 0:
        print(f"Error: El recorte para la imagen {image_name} está vacío.")
        return ss1_rescaled

    # Redimensionar la imagen al tamaño objetivo
    crop_resized = cv2.resize(crop, TARGET_SIZE)

    # Calcular las nuevas coordenadas reescaladas
    scale_x = TARGET_SIZE[0] / crop.shape[1]
    scale_y = TARGET_SIZE[1] / crop.shape[0]

    xmin_rescaled = int((x1 - x1_new) * scale_x)
    ymin_rescaled = int((y1 - y1_new) * scale_y)
    xmax_rescaled = int((x2 - x1_new) * scale_x)
    ymax_rescaled = int((y2 - y1_new) * scale_y)

    # Determinar el directorio base según el split
    if split.lower() in ['train', 'training']:
        base_dir = train_folder
    elif split.lower() == 'test':
        base_dir = test_folder
    else:
        print(f"Error: Split desconocido '{split}' para la imagen {image_name}.")
        return ss1_rescaled

    # Determinar la categoría de salida
    if finding_categories == 'Mass':
        output_dir = os.path.join(base_dir, 'masas')
    elif finding_categories == 'Suspicious Calcification':
        output_dir = os.path.join(base_dir, 'calcificaciones')
    elif finding_categories in otros_categories:
        output_dir = os.path.join(base_dir, 'otros')
    else:
        print(f"Error: Categoría desconocida '{finding_categories}' para la imagen {image_name}.")
        return ss1_rescaled

    # Mapear BIRADS a categoría
    birads_category = map_birads_to_category(birads)
    output_dir = os.path.join(output_dir, birads_category)

    # Guardar la imagen recortada
    roi_filename = f"{image_name}.png"
    output_path = os.path.join(output_dir, roi_filename)
    cv2.imwrite(output_path, crop_resized)
    print(f"Imagen ROI guardada en: {output_path}")

    # Guardar las coordenadas reescaladas
    new_row = ss1.loc[ss1['image_name'] == image_name].copy()
    new_row['xmin'] = xmin_rescaled
    new_row['ymin'] = ymin_rescaled
    new_row['xmax'] = xmax_rescaled
    new_row['ymax'] = ymax_rescaled
    ss1_rescaled = pd.concat([ss1_rescaled, new_row], ignore_index=True)

    return ss1_rescaled

# Función para procesar todas las imágenes
def process_all_images():
    global ss1_rescaled
    for _, row in ss1.iterrows():
        image_name = row['image_name']
        split = row['split']  # La columna 'split' tiene los valores 'train', 'training', o 'test'
        
        # Obtener 'birads' de 'finding_birads' o 'breast_birads'
        birads = row.get('finding_birads', '')
        if pd.isna(birads) or birads.strip() == '':
            birads = row.get('breast_birads', '')
            if pd.isna(birads) or birads.strip() == '':
                print(f"Advertencia: No se encontró 'finding_birads' ni 'breast_birads' para la imagen {image_name}. Asignando 'BI-RADS 0'.")
                birads = 'BI-RADS 0'
        
        finding_categories = row['finding_categories']

        # Procesar la imagen
        ss1_rescaled = extract_roi(image_name, split, birads, finding_categories, ss1_rescaled)

    # Guardar el nuevo CSV
    new_csv_path = '/Volumes/m2/Memoria/Code/PMM/VinDr-Mammo-Preprocessing/data/processed/csv/CROP8.csv'  # Cambiado a CROP7.csv
    ss1_rescaled.to_csv(new_csv_path, index=False)
    print(f"Nuevo CSV guardado en: {new_csv_path}")

# Ejecutar el procesamiento
process_all_images()


Imagen 001ade2a3cb53fd808bd2856a0df5413_0 tiene Photometric Interpretation: MONOCHROME1 (invertida)
Imagen ROI guardada en: /Volumes/m2/Memoria/Code/PMM/VinDr-Mammo-Preprocessing/data/processed/roi_images/ROICROP8/training/masas/sospechoso/001ade2a3cb53fd808bd2856a0df5413_0.png
Imagen 001ade2a3cb53fd808bd2856a0df5413_1 tiene Photometric Interpretation: MONOCHROME1 (invertida)
Imagen ROI guardada en: /Volumes/m2/Memoria/Code/PMM/VinDr-Mammo-Preprocessing/data/processed/roi_images/ROICROP8/training/calcificaciones/sospechoso/001ade2a3cb53fd808bd2856a0df5413_1.png
Imagen 001ade2a3cb53fd808bd2856a0df5413_2 tiene Photometric Interpretation: MONOCHROME1 (invertida)
Imagen ROI guardada en: /Volumes/m2/Memoria/Code/PMM/VinDr-Mammo-Preprocessing/data/processed/roi_images/ROICROP8/training/calcificaciones/sospechoso/001ade2a3cb53fd808bd2856a0df5413_2.png
Imagen 002074412a8fc178c271fb93b55c3e29_0 tiene Photometric Interpretation: MONOCHROME2
Imagen ROI guardada en: /Volumes/m2/Memoria/Code/PMM/Vi

/var/folders/ds/4z7lc9ld4mqb377_3t2v4lxm0000gn/T/ipykernel_1092/4008557562.py:195: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ss1_rescaled = pd.concat([ss1_rescaled, new_row], ignore_index=True)


Imagen 01599597388f3185563decc34945f6b3_0 tiene Photometric Interpretation: MONOCHROME2
Imagen ROI guardada en: /Volumes/m2/Memoria/Code/PMM/VinDr-Mammo-Preprocessing/data/processed/roi_images/ROICROP8/training/otros/sospechoso/01599597388f3185563decc34945f6b3_0.png
Imagen 0171ab32059f4c226164a13c311f6824_0 tiene Photometric Interpretation: MONOCHROME2
Imagen ROI guardada en: /Volumes/m2/Memoria/Code/PMM/VinDr-Mammo-Preprocessing/data/processed/roi_images/ROICROP8/training/calcificaciones/sospechoso/0171ab32059f4c226164a13c311f6824_0.png
Imagen 01958718afdf303581e758cdf34eaf8a_0 tiene Photometric Interpretation: MONOCHROME2
Imagen ROI guardada en: /Volumes/m2/Memoria/Code/PMM/VinDr-Mammo-Preprocessing/data/processed/roi_images/ROICROP8/test/masas/sospechoso/01958718afdf303581e758cdf34eaf8a_0.png
Imagen 019b9f6365fa641db040b5b643fadc42_0 tiene Photometric Interpretation: MONOCHROME2
Imagen ROI guardada en: /Volumes/m2/Memoria/Code/PMM/VinDr-Mammo-Preprocessing/data/processed/roi_images/

El script está diseñado para procesar imágenes DICOM de mamografías, extrayendo regiones de interés (ROIs) basadas en coordenadas proporcionadas en un archivo CSV. 

Primero, organiza las imágenes en categorías como masas, calcificaciones y otros, y subcategorías según los valores de BIRADS (benigno, maligno, sospechoso) dentro de carpetas de entrenamiento y prueba. 

Para cada imagen, el script localiza el archivo DICOM correspondiente, aplica ajustes como la inversión de la imagen si tiene una interpretación fotométrica MONOCHROME1, normaliza los valores de píxel, y realiza un recorte centrado con un padding del 10% en el lado mayor para mantener la proporción cuadrada. 

Posteriormente, redimensiona la ROI a un tamaño estándar de 299x299 píxeles y guarda tanto la imagen recortada como las nuevas coordenadas reescaladas en un archivo CSV (CROP8.csv). 

Este proceso asegura una organización estructurada y consistente de las ROIs, facilitando su uso para análisis posteriores o para el entrenamiento de modelos de aprendizaje automático.

Después de ejecutar el script de recorte, se realizó un conteo de las imágenes .png almacenadas en la carpeta ROICROP8 y sus subdirectorios.

Los resultados obtenidos muestran una distribución detallada por categorías y subcategorías dentro de los splits de Training y Test. 

En el split de Training, se contabilizaron 443 imágenes benignas, 90 malignas y 372 sospechosas en masas; 54 benignas, 33 malignas y 224 sospechosas en calcificaciones; y 215 benignas, 43 malignas y 187 sospechosas en otros. 

En el split de Test, las cifras fueron 123 benignas, 14 malignas y 81 sospechosas en masas; 11 benignas, 20 malignas y 60 sospechosas en calcificaciones; y 49 benignas, 14 malignas y 47 sospechosas en otros. 

En total, se contabilizaron 2080 imágenes .png 